In [1]:
!pip install transformers
!pip install evaluate
!pip install datasets
!pip install datasets[audio]


Defaulting to user installation because normal site-packages is not writeable
You should consider upgrading via the '/cluster/apps/nss/gcc-6.3.0/python/3.8.5/x86_64/bin/python3.8 -m pip install --upgrade pip' command.
Defaulting to user installation because normal site-packages is not writeable
You should consider upgrading via the '/cluster/apps/nss/gcc-6.3.0/python/3.8.5/x86_64/bin/python3.8 -m pip install --upgrade pip' command.
Defaulting to user installation because normal site-packages is not writeable


You should consider upgrading via the '/cluster/apps/nss/gcc-6.3.0/python/3.8.5/x86_64/bin/python3.8 -m pip install --upgrade pip' command.
Defaulting to user installation because normal site-packages is not writeable


You should consider upgrading via the '/cluster/apps/nss/gcc-6.3.0/python/3.8.5/x86_64/bin/python3.8 -m pip install --upgrade pip' command.


In [2]:
!module load gcc/8.2.0 python_gpu/3.10.4 r/4.0.2 git-lfs/2.3.0 eth_proxy npm/6.14.9 libsndfile/1.0.23


Due to MODULEPATH changes, the following have been reloaded:
  1) r/4.0.2

The following have been reloaded with a version change:
  1) cuda/11.0.3 => cuda/11.3.1        4) nccl/2.7.8-1 => nccl/2.11.4-1
  2) cudnn/8.0.5 => cudnn/8.2.1.32     5) openblas/0.2.20 => openblas/0.3.15
  3) gcc/6.3.0 => gcc/8.2.0            6) python_gpu/3.8.5 => python_gpu/3.10.4



In [3]:
import torch
from datasets import load_dataset
from datasets import DatasetDict
from datasets import Dataset
from transformers import AutoModelForSequenceClassification
import numpy as np
import pandas as pd
from transformers import TrainingArguments, Trainer
from sklearn.metrics import mean_absolute_error
from sklearn.metrics import mean_squared_error
from sklearn.metrics import r2_score
import numpy as np
from sklearn.model_selection import StratifiedKFold


In [5]:
# First make the kfold object
folds = StratifiedKFold(n_splits=5)

#create dataframes for train and test
train_df = pd.read_csv('preprocessed/train_full.csv')
#drop nan
train_df.dropna(inplace=True)
print(train_df.info())
folds_datasets = []
#get folds
for fold_, (train_index, test_index) in enumerate(folds.split(train_df, train_df['label'])):
    train_fold = train_df.iloc[train_index]
    test_fold = train_df.iloc[test_index]
    dataset_fold = DatasetDict({'train': Dataset.from_pandas(train_fold), 'test': Dataset.from_pandas(test_fold)})
    folds_datasets.append(dataset_fold)

<class 'pandas.core.frame.DataFrame'>
Int64Index: 2499150 entries, 0 to 2499999
Data columns (total 6 columns):
 #   Column                        Dtype 
---  ------                        ----- 
 0   tweet                         object
 1   label                         int64 
 2   clean_tweet                   object
 3   tokenized_tweet               object
 4   tokenized_tweet_no_stopwords  object
 5   text                          object
dtypes: int64(1), object(5)
memory usage: 133.5+ MB
None


In [6]:
#load dataset from DatasetDict
dataset = folds_datasets[0]


In [7]:
MAX_LEN=512
from transformers import AutoTokenizer

hugging_face_model = "cardiffnlp/twitter-roberta-base-sentiment-latest"
tokenizer = AutoTokenizer.from_pretrained(hugging_face_model)


def tokenize_function(examples):
    return tokenizer(examples["clean_tweet"], max_length=MAX_LEN, padding='max_length',)

def split_label(label):
    return [np.array(str(l).split(' '),dtype=float) for l in label]
dataset = dataset.map(lambda x: {"label": split_label(x["label"])}, batched=True)


Map:   0%|          | 0/1999320 [00:00<?, ? examples/s]

Map:   0%|          | 0/499830 [00:00<?, ? examples/s]

In [8]:
tokenized_datasets = dataset.map(tokenize_function, batched=True)

Map:   0%|          | 0/1999320 [00:00<?, ? examples/s]

Map:   0%|          | 0/499830 [00:00<?, ? examples/s]

In [9]:
tokenized_datasets['train'][0]['label']

[0.0]

In [10]:
tokenized_datasets=tokenized_datasets.filter(lambda x: len(x["input_ids"])<=180)

Filter:   0%|          | 0/1999320 [00:00<?, ? examples/s]

Filter:   0%|          | 0/499830 [00:00<?, ? examples/s]

In [11]:
tokenized_datasets

DatasetDict({
    train: Dataset({
        features: ['tweet', 'label', 'clean_tweet', 'tokenized_tweet', 'tokenized_tweet_no_stopwords', 'text', '__index_level_0__', 'input_ids', 'attention_mask'],
        num_rows: 1999320
    })
    test: Dataset({
        features: ['tweet', 'label', 'clean_tweet', 'tokenized_tweet', 'tokenized_tweet_no_stopwords', 'text', '__index_level_0__', 'input_ids', 'attention_mask'],
        num_rows: 499830
    })
})

In [12]:
tokenized_datasets["train"] = tokenized_datasets["train"].shuffle(seed=42)
#tokenized_datasets["validation"] = tokenized_datasets["validation"].shuffle(seed=42)
tokenized_datasets["test"] = tokenized_datasets["test"].shuffle(seed=42)

In [13]:
tokenized_datasets

DatasetDict({
    train: Dataset({
        features: ['tweet', 'label', 'clean_tweet', 'tokenized_tweet', 'tokenized_tweet_no_stopwords', 'text', '__index_level_0__', 'input_ids', 'attention_mask'],
        num_rows: 1999320
    })
    test: Dataset({
        features: ['tweet', 'label', 'clean_tweet', 'tokenized_tweet', 'tokenized_tweet_no_stopwords', 'text', '__index_level_0__', 'input_ids', 'attention_mask'],
        num_rows: 499830
    })
})

In [14]:
from transformers import DataCollatorWithPadding

tokenized_datasets.set_format('torch', columns=["input_ids", "attention_mask", "label"] )

data_collator = DataCollatorWithPadding(tokenizer=tokenizer)

In [15]:
import torch.nn as nn
from transformers import AutoModel,AutoConfig
from transformers.modeling_outputs import TokenClassifierOutput
class CustomModel(nn.Module):
    def __init__(self, checkpoint, num_labels ):
        super(CustomModel, self).__init__()
        self.num_labels = num_labels
        
        self.model = AutoModel.from_pretrained(checkpoint, config = AutoConfig.from_pretrained(checkpoint, output_hidden_state = True ) )
        self.dropout = nn.Dropout(0.1)
        self.classifier = nn.Linear(768, num_labels )
        
    def forward(self, input_ids = None, attention_mask=None, labels = None ):
        outputs = self.model(input_ids = input_ids, attention_mask = attention_mask  )
        
        last_hidden_state = outputs[0]
        sequence_outputs = self.dropout(last_hidden_state)
        
        logits = self.classifier(sequence_outputs[:, 0, : ].view(-1, 768 ))
        logits = torch.sigmoid(logits)
        loss = None
        if labels is not None:
            loss = torch.nn.functional.binary_cross_entropy(logits, labels)
            return TokenClassifierOutput(loss=loss, logits=logits, hidden_states=last_hidden_state)

In [16]:
from torch.utils.data import DataLoader

BATCH_SIZE = 32 
train_dataloader = DataLoader(
    tokenized_datasets['train'], shuffle = True, batch_size = BATCH_SIZE, collate_fn = data_collator
)

#eval_dataloader = DataLoader(
#    tokenized_datasets['validation'], shuffle = True, batch_size = BATCH_SIZE, collate_fn = data_collator
#)

test_dataloader = DataLoader(
    tokenized_datasets['test'], batch_size = 32, collate_fn = data_collator
)

In [17]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
torch.cuda.empty_cache()

model = CustomModel(checkpoint=hugging_face_model, num_labels=1).to(device)



Some weights of the model checkpoint at cardiffnlp/twitter-roberta-base-sentiment-latest were not used when initializing RobertaModel: ['classifier.dense.bias', 'classifier.dense.weight', 'classifier.out_proj.weight', 'classifier.out_proj.bias']
- This IS expected if you are initializing RobertaModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing RobertaModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


In [18]:
torch.cuda.get_device_name(device)

'NVIDIA GeForce RTX 2080 Ti'

In [19]:
for name, param in model.named_parameters():
  if "encoder.layer.0"  in name:
    param.requires_grad=True
  elif "encoder.layer.1."  in name:
    param.requires_grad=True
  elif "encoder.layer.2"  in name:
    param.requires_grad=True
  elif "encoder.layer.3"  in name:
    param.requires_grad=True
  elif "encoder.layer.4"  in name:
    param.requires_grad=True
  elif "encoder.layer.5"  in name:
    param.requires_grad=True
  elif "encoder.layer.6"  in name:
    param.requires_grad=True
  elif "encoder.layer.7"  in name:
    param.requires_grad=True
  elif "encoder.layer.8"  in name:
    param.requires_grad=True
  elif "encoder.layer.9"  in name:
    param.requires_grad=True
  elif "encoder.layer.10"  in name:
    param.requires_grad=True
  elif "encoder.layer.11"  in name:
    param.requires_grad=True
  elif "embeddings"  in name:
    param.requires_grad=True
  else:
    print(name)

model.pooler.dense.weight
model.pooler.dense.bias
classifier.weight
classifier.bias


In [20]:
from transformers import AdamW, get_scheduler

optimizer = AdamW(model.parameters(), lr = 5e-5 )

num_epoch = 1

num_training_steps = num_epoch * len(train_dataloader)

lr_scheduler = get_scheduler(
    'linear',
    optimizer = optimizer,
    num_warmup_steps=0,
    num_training_steps = num_training_steps,
    
)

/cluster/home/fgirlanda/.local/lib/python3.8/site-packages/transformers/optimization.py:411: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(


In [22]:
from tqdm.auto import tqdm

progress_bar_train = tqdm(range(num_training_steps))

for epoch in range(num_epoch):
    model.train()
    for batch in train_dataloader:
        batch = { k: v.to(device) for k, v in batch.items() }
        outputs = model(**batch)
        loss = outputs.loss
        loss.backward()
        
        optimizer.step()
        lr_scheduler.step()
        optimizer.zero_grad()
        progress_bar_train.update(1)

  0%|          | 0/62479 [00:00<?, ?it/s]

You're using a RobertaTokenizerFast tokenizer. Please note that with a fast tokenizer, using the `__call__` method is faster than using a method to encode the text followed by a call to the `pad` method to get a padded encoding.


In [23]:
threshold = 0.5
correct_predictions = 0
total_predictions = 0

model.eval()
progress_bar_test = tqdm(range(num_epoch * len(test_dataloader) ))

metric_count = 0
for batch in test_dataloader:
    batch = { k: v.to(device) for k, v in batch.items() }
    with torch.no_grad():
        outputs = model(**batch)
        
    logits = outputs.logits
    metric_count += torch.nn.functional.binary_cross_entropy(logits, batch['labels'])
    
    predictions = (logits >= threshold).int()  
    correct_predictions += (predictions == batch['labels']).sum().item()
    total_predictions += len(batch['labels'])

    progress_bar_test.update(1)
print("TEST ACC: ", correct_predictions / total_predictions)
print("TEST BCE: ", metric_count / len(test_dataloader))

  0%|          | 0/15620 [00:00<?, ?it/s]

TEST ACC:  0.5000260088430066
TEST BCE:  tensor(0.7389, device='cuda:0')


In [25]:
torch.save(model.state_dict(), "roBERTa_12_layers_with_emb_light_pre")